## UAV-Cyber-SIM

In [1]:
import random
from collections import defaultdict

from config import Color
from helpers import clean
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigQGC,
    ConfigVis,
    Gazebo,
    NoneVisualizer,
    Simulator,
)

clean()

## Create Plans

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

gcs=[Color.GREEN,Color.ORANGE,Color.RED]
n_uavs_per_gcs = 25
side_len = 10
altitude = 5
max_delay = 10 #sec

base_homes = ENUPose.list([
    (i * 50, j * 3 * side_len, 0, 0)
    for i in range(len(gcs))
    for j in range(n_uavs_per_gcs)
])

base_paths = [
    Plan.create_square_path(side_len=side_len, alt=altitude,heading=0) 
    for _ in base_homes
    ]

colors = [color for color in gcs for _ in range(n_uavs_per_gcs)]

msn_delays=[random.randint(0, max_delay) for _ in base_homes]
msn_delays=[1 for _ in base_homes]


## Assign vehicles to GCS (by color)
gcs_sysids: dict[str, list[int]] = defaultdict(list)

for i,color in enumerate(colors,start=1):
    gcs_sysids[f'{color.name} {color.emoji}'].append(i)

## Gazebo Congiguration

In [4]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway3.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
#gaz_config.show()

## QGroundControl Configuration

In [5]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

# No Visualizer

In [6]:
novis_config = ConfigVis[int]()

for i,_ in enumerate(base_homes):
    novis_config.add_vehicle(i)

## Visualization Parameters

In [7]:
novis=NoneVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulator

In [8]:
simulator = Simulator(
	visualizers=[qgc],
	gcs_sysids=gcs_sysids,
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)

orac = simulator.launch()

🚀 ArduPilot SITL vehicle 1 launched (PID 1104505)
🚀 ArduPilot SITL vehicle 3 launched (PID 1104509)
🚀 ArduPilot SITL vehicle 2 launched (PID 1104507)
🚀 ArduPilot SITL vehicle 5 launched (PID 1104513)
🚀 ArduPilot SITL vehicle 7 launched (PID 1104517)
🚀 ArduPilot SITL vehicle 4 launched (PID 1104512)
🚀 ArduPilot SITL vehicle 8 launched (PID 1104519)
🚀 UAV logic for vehicle 1 launched (PID 1104520)
🚀 ArduPilot SITL vehicle 6 launched (PID 1104515)
🚀 UAV logic for vehicle 3 launched (PID 1104521)
🚀 UAV logic for vehicle 5 launched (PID 1104531)
🚀 Proxy for vehicle 1 launched (PID 1104540)
🚀 ArduPilot SITL vehicle 12 launched (PID 1104554)
🚀 UAV logic for vehicle 2 launched (PID 1104523)
🚀 ArduPilot SITL vehicle 10 launched (PID 1104533)
🚀 UAV logic for vehicle 8 launched (PID 1104549)
🚀 ArduPilot SITL vehicle 9 launched (PID 1104525)
🚀 UAV logic for vehicle 4 launched (PID 1104536)
🚀 UAV logic for vehicle 6 launched (PID 1104545)
🚀 UAV logic for vehicle 7 launched (PID 1104535)
🚀 ArduPilot

## Oracle checking

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 34: ✅ Sent mission item 3
Vehicle 37: ✅ Sent mission item 0
Vehicle 34: ✅ Sent mission item 4
Vehicle 37: ✅ Sent mission item 1
Vehicle 34: ✅ Sent mission item 5
Vehicle 37: ✅ Sent mission item 2
Vehicle 34: ✅ Sent mission item 6
Vehicle 37: ✅ Sent mission item 3
Vehicle 34: ✅ Sent mission item 7
Vehicle 60: ✅ Sent mission item 0
Vehicle 37: ✅ Sent mission item 4
Vehicle 53: ✅ Sent mission item 0
Vehicle 34: ✅ Mission upload successful!
Vehicle 60: ✅ Sent mission item 1
Vehicle 37: ✅ Sent mission item 5
Vehicle 34: ✅ Action Done: 📤 UPLOAD_MISSION
Vehicle 53: ✅ Sent mission item 1
Vehicle 60: ✅ Sent mission item 2
Vehicle 35: ✅ Sent mission item 0
Vehicle 37: ✅ Sent mission item 6
Vehicle 47: ✅ Sent mission item 0
Vehicle 53: ✅ Sent mission item 2
Vehicle 60: ✅ Sent mission item 3
Vehicle 40: ✅ Sent mission item 0
Vehicle 35: ✅ Sent mission item 1
Vehicle 37: ✅ Sent mission item 7
Vehicle 56: ✅ Sent mission item 0
Vehicle 47: ✅ Sent mission item 1
Vehicle 53: ✅ Sent mission item

📤 GCS ← UAV 75: Sending DONE (attempt 20)
📤 GCS ← UAV 70: Sending DONE (attempt 3)
✅ ACK received. DONE message acknowledged.
❎ Vehicle 70 logic stopped.
Connection reset or closed by peer on TCP socket
📤 GCS ← UAV 75: Sending DONE (attempt 21)
❎ Proxy 73 stopped.
📤 GCS ← UAV 75: Sending DONE (attempt 22)
❎ Proxy 70 stopped.
📤 GCS ← UAV 75: Sending DONE (attempt 23)
📤 GCS ← UAV 75: Sending DONE (attempt 24)
📤 GCS ← UAV 75: Sending DONE (attempt 25)
📤 GCS ← UAV 75: Sending DONE (attempt 26)
📤 GCS ← UAV 75: Sending DONE (attempt 27)
📤 GCS ← UAV 75: Sending DONE (attempt 28)
📤 GCS ← UAV 75: Sending DONE (attempt 29)
📤 GCS ← UAV 75: Sending DONE (attempt 30)
📤 GCS ← UAV 75: Sending DONE (attempt 31)
📤 GCS ← UAV 75: Sending DONE (attempt 32)
📤 GCS ← UAV 75: Sending DONE (attempt 33)
✅ ACK received. DONE message acknowledged.
❎ Vehicle 75 logic stopped.
Connection reset or closed by peer on TCP socket
❎ Proxy 75 stopped.
